## 1. MNIST학습하기
- 미니배치를 사용함
- 미니배치 : 데이터를 적당한 크기로 잘라서 학습시킴

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./mnist/data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [2]:
X=tf.placeholder(tf.float32,[None,784]) # None은 배치사이즈 , None으로 설정시 텐서플로가 알아서 계산해서 적용
Y=tf.placeholder(tf.float32,[None,10])

In [3]:
W1=tf.Variable(tf.random_normal([784,256],stddev=0.01)) # 표준편차가 0.01인 정규분포를 가지는 임의의값으로 뉴런을 초기화
L1=tf.nn.relu(tf.matmul(X,W1))

W2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
L2=tf.nn.relu(tf.matmul(L1,W2))

W3=tf.Variable(tf.random_normal([256,10],stddev=0.01))
model=tf.matmul(L2,W3)

In [4]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer=tf.train.AdamOptimizer(0.001).minimize(cost)

In [5]:
init=tf.global_variables_initializer() # 가중치 초기화
sess=tf.Session()
sess.run(init)

In [13]:
#학습진행
batch_size=100
total_batch=int(mnist.train.num_examples/batch_size) # 데이터수/배치크기 = 미니배치크기
for epoch in range(15):
    total_cost=0
    for i in range(total_batch):
        batch_x,batch_y=mnist.train.next_batch(batch_size)
        
        _,cost_val=sess.run([optimizer,cost],feed_dict={X:batch_x,Y:batch_y})
        
        # 모델출력값 확인
        #print("result: ",sess.run(model,feed_dict={X:batch_x}))
        
        total_cost+=cost_val
    print('Epoch:','%04d'%(epoch+1),
        'Avg cost: ','{:.3f}'.format(total_cost/total_batch))

# test
print("predict-",sess.run(tf.argmax(model,1),feed_dict={X:mnist.test.images}),"target-",tf.argmax(mnist.test.labels,1))
is_correct=tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))
print('정확도:',sess.run(accuracy,feed_dict={X:mnist.test.images,Y:mnist.test.labels}))

Epoch: 0001 Avg cost:  0.006
Epoch: 0002 Avg cost:  0.005
Epoch: 0003 Avg cost:  0.004
Epoch: 0004 Avg cost:  0.001
Epoch: 0005 Avg cost:  0.004
Epoch: 0006 Avg cost:  0.011
Epoch: 0007 Avg cost:  0.006
Epoch: 0008 Avg cost:  0.002
Epoch: 0009 Avg cost:  0.002
Epoch: 0010 Avg cost:  0.004
Epoch: 0011 Avg cost:  0.005
Epoch: 0012 Avg cost:  0.003
Epoch: 0013 Avg cost:  0.005
Epoch: 0014 Avg cost:  0.007
Epoch: 0015 Avg cost:  0.001
predict- [7 2 1 ... 4 5 6] target- Tensor("ArgMax_4:0", shape=(10000,), dtype=int64)
정확도: 0.9828


## 2. Dropout 사용해보기
- Dropout은 과적합을 방지하기 위한 기법 중 하나이다.
- 학습 시 전체 신경망 중 일부만을 사용하도록 하는 것. (일부 뉴런을 제거)
- 학습 시 일부 뉴런을 학습시키지 않기 때문에 신경망의 충분한 학습시간이 조금 더 오래걸린다.

- **주의해야할 점**
    - 학습이 끝난 후 예측시에는 신경망 전체를 사용하도록 해야한다.

In [14]:
W1=tf.Variable(tf.random_normal([784,256],stddev=0.01))
L1=tf.nn.relu(tf.matmul(X,W1))
L1=tf.nn.dropout(L1,0.8)#80%의 뉴런만 사용하겠다는 것

W2=tf.Variable(tf.random_normal([256,256],stddev=0.01))
L2=tf.nn.relu(tf.matmul(L1,W2))
L2=tf.nn.dropout(L2,0.8)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
